In [ ]:
import os
import pandas as pd

Read annotated CSV file

In [ ]:
annotated_df = pd.read_csv('../data/DataExp4_filtered_test_val_annotated.csv')
original_annotated_df = pd.read_csv('../data/DataExp4_filtered_test_val_annotated.csv')

In [ ]:
not_annotated_df = pd.read_csv('../data/DataExp4_filtered_not_annotated.csv')

In [ ]:
if "Unnamed: 0" in annotated_df.columns:
    annotated_df.drop(["Unnamed: 0"], axis=1, inplace=True)
if "Unnamed: 0" in original_annotated_df.columns:
    original_annotated_df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [ ]:
if "Unnamed: 0" in not_annotated_df.columns:
    not_annotated_df.drop(["Unnamed: 0"], axis=1, inplace=True)

Fill the fixed title and fixed query body with the fixed ones, if available

In [ ]:
annotated_df["Fixed Title"].fillna(annotated_df["Title"], inplace=True)
annotated_df["Fixed QueryBody"].fillna(annotated_df["QueryBody"], inplace=True)

In [ ]:
not_annotated_df["Fixed Title"].fillna(not_annotated_df["Title"], inplace=True)
not_annotated_df["Fixed QueryBody"].fillna(not_annotated_df["QueryBody"], inplace=True)

In [ ]:
annotated_df["Type of Fix"].fillna("", inplace=True)
annotated_df["Type of Fix"].unique()

In [ ]:
original_annotated_df["Type of Fix"].fillna("", inplace=True)
original_annotated_df["Type of Fix"].unique()

In [ ]:
annotated_df["Type of Fix"] = annotated_df["Type of Fix"].apply(lambda x: x.upper())

In [ ]:
original_annotated_df["Type of Fix"] = original_annotated_df["Type of Fix"].apply(lambda x: x.upper())

Remove rows without annotation or rows that couldn't be fixed

In [ ]:
filtered_annotated_df = annotated_df[~annotated_df["Type of Fix"].isin({"NIC", "NINC", "NEC", "INC"})].copy()

In [ ]:
filtered_annotated_df

In [ ]:
original_annotated_df = original_annotated_df[~original_annotated_df["Type of Fix"].isin({"NIC", "NINC", "NEC", "INC"})].copy()

In [ ]:
original_annotated_df

In [ ]:
filtered_annotated_df["validated"] = (filtered_annotated_df["Type of Fix"].fillna('') != '').astype(bool)

In [ ]:
original_annotated_df["validated"] = (original_annotated_df["Type of Fix"].fillna('') != '').astype(bool)

Delete unused columns

In [ ]:
del filtered_annotated_df["Title"]
del filtered_annotated_df["QueryBody"]
del filtered_annotated_df["Type of Fix"]
del filtered_annotated_df["Fixer"]
del filtered_annotated_df["Faruk_Error"]

In [ ]:
del annotated_df["Title"]
del annotated_df["QueryBody"]
del annotated_df["Type of Fix"]
del annotated_df["Fixer"]
del annotated_df["Faruk_Error"]

In [ ]:
del original_annotated_df["Fixed Title"]
del original_annotated_df["Fixed QueryBody"]
del original_annotated_df["Type of Fix"]
del original_annotated_df["Fixer"]
del original_annotated_df["Faruk_Error"]

In [ ]:
del not_annotated_df["Title"]
del not_annotated_df["QueryBody"]
del not_annotated_df["Type of Fix"]
del not_annotated_df["Fixer"]
del not_annotated_df["Faruk_Error"]

Rename columns back again

In [ ]:
filtered_annotated_df.rename(columns={"Fixed Title": "Title", "Fixed QueryBody": "QueryBody"}, inplace=True)
annotated_df.rename(columns={"Fixed Title": "Title", "Fixed QueryBody": "QueryBody"}, inplace=True)
not_annotated_df.rename(columns={"Fixed Title": "Title", "Fixed QueryBody": "QueryBody"}, inplace=True)

Reorder columns

In [ ]:
cols = ["QuerySetId", "Title", "Description", "QueryBody", "CreationDate", "validated"]

In [ ]:
filtered_annotated_df = filtered_annotated_df[cols]
original_annotated_df = original_annotated_df[cols]

In [ ]:
not_annotated_df["validated"] = False
annotated_df["validated"] = False

In [ ]:
annotated_df = annotated_df[cols]

In [ ]:
not_annotated_df = not_annotated_df[cols]

Check final dataframe before saving it to disk

In [ ]:
filtered_annotated_df

In [ ]:
not_annotated_df

In [ ]:
annotated_df

In [ ]:
original_annotated_df

In [ ]:
filtered_annotated_df.to_csv("../data/DataExp4_filtered_fixed.csv", index=False)

Split into train/val/test sets

In [ ]:
val_test_df = filtered_annotated_df[filtered_annotated_df["validated"]]
original_val_test_df = original_annotated_df[original_annotated_df["validated"]]

In [ ]:
train_df = not_annotated_df.copy()

In [ ]:
val_test_query_set_ids = val_test_df["QuerySetId"].tolist()
train_query_set_ids = train_df["QuerySetId"].tolist()
val_test_query_set_ids_in_train = [sql for sql in val_test_query_set_ids if sql in train_query_set_ids]
print(f"Number of QuerySetId in val/test that appear in train: {len(val_test_query_set_ids_in_train)}")

In [ ]:
val_df = val_test_df.iloc[:int(val_test_df.shape[0] / 2)]
test_df = val_test_df.iloc[int(val_test_df.shape[0] / 2):]

In [ ]:
original_val_df = original_val_test_df.iloc[:int(original_val_test_df.shape[0] / 2)]
original_test_df = original_val_test_df.iloc[int(original_val_test_df.shape[0] / 2):]

In [ ]:
if not os.path.exists("../data/sede/"):
    os.makedirs("../data/sede/")

In [ ]:
print(f"Train set size: {len(train_df)}")
print(f"Val set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")
print(f"Original Val set size: {len(original_val_df)}")
print(f"Original Test set size: {len(original_test_df)}")

In [ ]:
train_df.to_json("../data/sede/train.jsonl", orient="records", lines=True)
val_df.to_json("../data/sede/val.jsonl", orient="records", lines=True)
original_val_df.to_json("../data/sede/val_original.jsonl", orient="records", lines=True)
test_df.to_json("../data/sede/test.jsonl", orient="records", lines=True)
original_test_df.to_json("../data/sede/test_original.jsonl", orient="records", lines=True)